# 02 - Data Preprocessing for Clustering

## Overview
This notebook prepares the data for clustering analysis by handling missing values, feature selection, scaling, and dimensionality reduction.

**Prerequisites:**
- Complete `01_Data_Exploration_EDA.ipynb` first
- Dataset: `../Data/eda_complete_dataset.csv`

**Objectives:**
- Feature engineering and selection
- Handle outliers and missing values
- Scale and normalize features
- Apply dimensionality reduction techniques
- Prepare data for clustering algorithms

**Outputs:**
- Preprocessed dataset ready for clustering
- Scaled features
- PCA and t-SNE transformed data
- Feature importance analysis

## 1. Library Imports and Setup

In [1]:
# Standard Data Manipulation
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# Preprocessing and Scaling
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# Dimensionality Reduction
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

# Feature Selection
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest, f_classif

# Outlier Detection
from sklearn.ensemble import IsolationForest
from sklearn.covariance import EllipticEnvelope

# Utilities
import warnings
warnings.filterwarnings('ignore')

# Set style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("Preprocessing libraries imported successfully!")

Preprocessing libraries imported successfully!


## 2. Load Preprocessed Data from EDA

In [2]:
# Load the dataset from EDA notebook
try:
    data = pd.read_csv('../Data/eda_complete_dataset.csv')
    print(f"Dataset loaded successfully: {data.shape}")
except FileNotFoundError:
    print("EDA dataset not found. Please run 01_Data_Exploration_EDA.ipynb first.")
    # Fallback to original dataset
    data = pd.read_csv('../Data/company_esg_financial_dataset.csv')
    print(f"Using original dataset: {data.shape}")

print(f"Columns: {list(data.columns)}")
data.head()

Dataset loaded successfully: (11000, 16)
Columns: ['CompanyID', 'CompanyName', 'Industry', 'Region', 'Year', 'Revenue', 'ProfitMargin', 'MarketCap', 'GrowthRate', 'ESG_Overall', 'ESG_Environmental', 'ESG_Social', 'ESG_Governance', 'CarbonEmissions', 'WaterUsage', 'EnergyConsumption']


,CompanyID,CompanyName,Industry,Region,Year,Revenue,ProfitMargin,MarketCap,GrowthRate,ESG_Overall,ESG_Environmental,ESG_Social,ESG_Governance,CarbonEmissions,WaterUsage,EnergyConsumption
0,1,Company_1,Retail,Latin America,2015,459.2,6.0,337.5,NaN,57.0,60.7,33.5,76.8,35577.4,17788.7,71154.7
1,1,Company_1,Retail,Latin America,2016,473.8,4.6,366.6,3.2,56.7,58.9,32.8,78.5,37314.7,18657.4,74629.4
2,1,Company_1,Retail,Latin America,2017,564.9,5.2,313.4,19.2,56.5,57.6,34.0,77.8,45006.4,22503.2,90012.9
3,1,Company_1,Retail,Latin America,2018,558.4,4.3,283.0,-1.1,58.0,62.3,33.4,78.3,42650.1,21325.1,85300.2
4,1,Company_1,Retail,Latin America,2019,554.5,4.9,538.1,-0.7,56.6,63.7,30.0,76.1,41799.4,20899.7,83598.8


## 3. Feature Engineering and Selection

In [3]:
# Define feature categories
financial_features = ['Revenue', 'ProfitMargin', 'MarketCap', 'GrowthRate']
esg_features = ['ESG_Overall', 'ESG_Environmental', 'ESG_Social', 'ESG_Governance']
environmental_features = ['CarbonEmissions', 'WaterUsage', 'EnergyConsumption']
categorical_features = ['Industry', 'Region']
identifier_features = ['CompanyID', 'CompanyName', 'Year']

# Combine numerical features for clustering
numerical_features = financial_features + esg_features + environmental_features

print(f"Financial features: {financial_features}")
print(f"ESG features: {esg_features}")
print(f"Environmental features: {environmental_features}")
print(f"Categorical features: {categorical_features}")
print(f"\nTotal numerical features for clustering: {len(numerical_features)}")

Financial features: ['Revenue', 'ProfitMargin', 'MarketCap', 'GrowthRate']
ESG features: ['ESG_Overall', 'ESG_Environmental', 'ESG_Social', 'ESG_Governance']
Environmental features: ['CarbonEmissions', 'WaterUsage', 'EnergyConsumption']
Categorical features: ['Industry', 'Region']

Total numerical features for clustering: 11


## 4. Handle Missing Values and Outliers

In [4]:
# Check for missing values in numerical features
missing_check = data[numerical_features].isnull().sum()
print("Missing values in numerical features:")
print(missing_check[missing_check > 0])

# Handle missing values if any
if missing_check.sum() > 0:
    # Fill missing values with median for numerical features
    for feature in numerical_features:
        if data[feature].isnull().sum() > 0:
            median_value = data[feature].median()
            data[feature].fillna(median_value, inplace=True)
            print(f"Filled {feature} missing values with median: {median_value:.2f}")
else:
    print("No missing values found in numerical features.")

Missing values in numerical features:
GrowthRate    1000
dtype: int64
Filled GrowthRate missing values with median: 4.90


## 5. Feature Scaling and Normalization

In [5]:
# Prepare the data for scaling
X = data[numerical_features].copy()

# Apply different scaling methods
scalers = {
    'StandardScaler': StandardScaler(),
    'MinMaxScaler': MinMaxScaler(),
    'RobustScaler': RobustScaler()
}

scaled_data = {}

for name, scaler in scalers.items():
    scaled_features = scaler.fit_transform(X)
    scaled_data[name] = pd.DataFrame(scaled_features, columns=numerical_features, index=X.index)
    print(f"{name} applied successfully.")

# Use StandardScaler as default for clustering
X_scaled = scaled_data['StandardScaler']
print(f"\nUsing StandardScaler for clustering analysis.")
print(f"Scaled data shape: {X_scaled.shape}")

StandardScaler applied successfully.
MinMaxScaler applied successfully.
RobustScaler applied successfully.

Using StandardScaler for clustering analysis.
Scaled data shape: (11000, 11)


## 6. Dimensionality Reduction

In [6]:
# Apply PCA
pca = PCA()
X_pca = pca.fit_transform(X_scaled)

# Calculate cumulative explained variance
cumsum_var = np.cumsum(pca.explained_variance_ratio_)

# Find number of components for 95% variance
n_components_95 = np.argmax(cumsum_var >= 0.95) + 1

print(f"Number of components for 95% variance: {n_components_95}")
print(f"Explained variance by first 3 components: {cumsum_var[2]:.3f}")

# Create PCA dataframe with optimal components
pca_optimal = PCA(n_components=n_components_95)
X_pca_optimal = pca_optimal.fit_transform(X_scaled)

pca_columns = [f'PCA_{i+1}' for i in range(n_components_95)]
X_pca_df = pd.DataFrame(X_pca_optimal, columns=pca_columns, index=X_scaled.index)

print(f"PCA transformed data shape: {X_pca_df.shape}")

Number of components for 95% variance: 7
Explained variance by first 3 components: 0.688
PCA transformed data shape: (11000, 7)


In [7]:
# Apply t-SNE for visualization (using first few PCA components)
tsne = TSNE(n_components=2, random_state=42, perplexity=30)
X_tsne = tsne.fit_transform(X_pca_optimal[:, :5])  # Use first 5 PCA components

X_tsne_df = pd.DataFrame(X_tsne, columns=['t-SNE_1', 't-SNE_2'], index=X_scaled.index)

print(f"t-SNE transformed data shape: {X_tsne_df.shape}")

t-SNE transformed data shape: (11000, 2)


## 7. Save Preprocessed Data

In [8]:
# Combine all preprocessed data
preprocessed_data = data.copy()

# Add scaled features
for col in X_scaled.columns:
    preprocessed_data[f'{col}_scaled'] = X_scaled[col]

# Add PCA features
for col in X_pca_df.columns:
    preprocessed_data[col] = X_pca_df[col]

# Add t-SNE features
for col in X_tsne_df.columns:
    preprocessed_data[col] = X_tsne_df[col]

# Save datasets
preprocessed_data.to_csv('../Data/preprocessed_complete_dataset.csv', index=False)
X_scaled.to_csv('../Data/scaled_features.csv', index=False)
X_pca_df.to_csv('../Data/pca_features.csv', index=False)
X_tsne_df.to_csv('../Data/tsne_features.csv', index=False)

print("Preprocessed datasets saved:")
print("- ../Data/preprocessed_complete_dataset.csv")
print("- ../Data/scaled_features.csv")
print("- ../Data/pca_features.csv")
print("- ../Data/tsne_features.csv")

print(f"\nReady for clustering analysis!")
print(f"Scaled features shape: {X_scaled.shape}")
print(f"PCA features shape: {X_pca_df.shape}")
print(f"t-SNE features shape: {X_tsne_df.shape}")

Preprocessed datasets saved:
- ../Data/preprocessed_complete_dataset.csv
- ../Data/scaled_features.csv
- ../Data/pca_features.csv
- ../Data/tsne_features.csv

Ready for clustering analysis!
Scaled features shape: (11000, 11)
PCA features shape: (11000, 7)
t-SNE features shape: (11000, 2)
